# Enriquecimiento de predicciones ARIMA con datos reales de ventas

Este notebook cruza las predicciones generadas por un modelo ARIMA con los datos históricos del archivo `sell-in.txt`, específicamente para el período `201912`.

El objetivo es comparar o enriquecer la predicción con la venta real (`tn`) observada ese mes.


In [12]:

import pandas as pd


## Carga de archivos

In [13]:

# Definir rutas
PATH_FORECAST = "C:\\Developer\\Laboratorio_III\\data\\forecast_product_phase1.csv"
PATH_SELLIN = "C:\\Developer\\Laboratorio_III\\data\\sell-in.txt"

# Cargar predicciones
df_pred = pd.read_csv(PATH_FORECAST)

# Cargar ventas
df_sellin = pd.read_csv(PATH_SELLIN, sep="\t")
df_sellin['periodo'] = df_sellin['periodo'].astype(str)


## Cruce de predicción con datos reales del período 201912

In [15]:

# Filtrar ventas de diciembre 2019
df_dic = df_sellin[df_sellin['periodo'] == '201912']

# Agregar ventas por producto
ventas_diciembre = df_dic.groupby('product_id')['tn'].sum().reset_index()
ventas_diciembre = ventas_diciembre.rename(columns={'tn': 'tn_real'})

# Cruce con predicciones
df_enriquecido = df_pred.merge(ventas_diciembre, on='product_id', how='left')


## Guardado del archivo enriquecido

In [16]:

df_enriquecido.to_csv("forecast_product_phase1_con_real.csv", index=False)
print("Archivo generado: forecast_product_phase1_con_real.csv")


Archivo generado: forecast_product_phase1_con_real.csv


## Visualización interactiva: Predicción vs. Real

Generamos un gráfico de dispersión (`scatter plot`) donde:

- Eje X: predicción del modelo (`tn`)
- Eje Y: venta real observada (`tn_real`)

Además, incorporamos un filtro interactivo para seleccionar uno o varios `product_id` y analizar sus resultados individualmente.


In [17]:

import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Widget interactivo de selección de productos
productos_unicos = df_enriquecido['product_id'].unique()
selector = widgets.SelectMultiple(
    options=sorted(productos_unicos),
    value=sorted(productos_unicos)[:10],
    description='Productos',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

# Función de visualización
def plot_scatter(productos):
    datos = df_enriquecido[df_enriquecido['product_id'].isin(productos)]
    fig = px.scatter(
        datos,
        x="tn", y="tn_real",
        hover_name="product_id",
        title="Predicción vs. Real (por producto)",
        labels={"tn": "Predicción (tn)", "tn_real": "Real (tn)"}
    )
    fig.add_shape(
        type='line',
        x0=0, y0=0, x1=datos['tn'].max(), y1=datos['tn'].max(),
        line=dict(color='gray', dash='dash')
    )
    fig.show()

# Mostrar selector + gráfico
widgets.interact(plot_scatter, productos=selector);


interactive(children=(SelectMultiple(description='Productos', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), layout=Lay…